In [175]:
import sqlparse
import networkx as nx
import re

In [180]:
sql_content = open("./01_02_Claims.sql").read()

In [183]:
sql_statements = sqlparse.split(sql_content)
for sql_statement in sql_statements:
    print(sqlparse.parse(sql_statement))
    query_lines = []
    parsed_statement = sqlparse.parse(sql_statement)[0]
    for sql_line in parsed_statement.value.split("\n"):
        if not sql_line.startswith("#"):
            query_lines.append(" ".join(sql_line.split()))
    single_line_query = " ".join(query_lines).replace("[\s]+", " ")
    print((parsed_statement.get_type().lower()))
    print(single_line_query)
    print("--")

(<Statement '# Make...' at 0x7F277AE39480>,)
unknown
create table `{{params.reporting_database}}`.`claim` like `{{params.raw_database}}`.`claim`;
--
(<Statement 'insert...' at 0x7F277AE39930>,)
insert
insert into `{{params.reporting_database}}`.`claim` select * FROM `{{params.raw_database}}`.`claim`;
--
()


IndexError: tuple index out of range

# Dependency Generation

In [131]:
all_sql_content = open("./generate_database.sql").read()

sql_statements = sqlparse.split(all_sql_content)

In [132]:
all_insert_queries = []
for sql_statement in sql_statements:
    query_lines = []
    parsed_statement = sqlparse.parse(sql_statement)[0]
    for sql_line in parsed_statement.value.split("\n"):
        if not sql_line.startswith("#"):
            query_lines.append(" ".join(sql_line.split()))
    if len(query_lines) < 1:
        continue
    single_line_query = " ".join(query_lines).replace("[\s]+", " ").replace(
        "PatentsView_20171226_v2", "PatentsView").replace(
            "patent_20171226", "patent")
    if parsed_statement.get_type().lower() == 'ddl':
        if single_line_query.startswith("DROP"):
            continue
    if parsed_statement.get_type().lower() == 'insert':
        all_insert_queries.append(single_line_query)

In [133]:
G=nx.DiGraph()

In [164]:
dependencies = []
for insert_query in all_insert_queries:
    insert_destination_search = re.search("INSERT INTO\s+(`[^`]+`\.`[^`]+`)",
                                          insert_query, re.IGNORECASE)
    if insert_destination_search is None:
        print(insert_query)
    destination = insert_destination_search.groups(1)[0]
#    print(destination)
    table_finder = re.compile("`[^`]+`\.`[^`]+`",  re.IGNORECASE)
    tables = table_finder.findall(insert_query)
    G.add_node(
        destination,attr_dict= {
            "sql":
            re.sub(
                "`PatentsView`(\.`[^`]+`)",
                "`{{reporting_database}}`\1",
                re.sub(
                    "`patent`(\.`[^`]+`)",
                    "`{{raw_database}}`\1",
                    insert_query,
                    flags=re.IGNORECASE),
                flags=re.IGNORECASE)
        })
    if destination=='`PatentsView`.`cpc_current_copy`':
            print(insert_query)
    for table in tables:
        if destination=='`PatentsView`.`cpc_current_copy`':
            print(table)
        if table != destination:
            G.add_edge(table, destination)

insert into `PatentsView`.`cpc_current_copy` select distinct patent_id,sequence,section_id,subsection_id,group_id,subgroup_id,category from `PatentsView`.`cpc_current`;
`PatentsView`.`cpc_current_copy`
`PatentsView`.`cpc_current`


In [167]:
nx.connected_components(G)

NetworkXNotImplemented: not implemented for directed type

In [116]:
sorted(G.in_degree, key=lambda x: x[1])

[('`patent`.`patent_assignee`', 0),
 ('`patent`.`patent_inventor`', 0),
 ('`patent`.`patent_lawyer`', 0),
 ('`patent`.`rawexaminer`', 0),
 ('`patent`.`rawlocation`', 0),
 ('`patent`.`patent`', 0),
 ('`patent`.`rawassignee`', 0),
 ('`patent`.`location`', 0),
 ('`patent`.`rawinventor`', 0),
 ('`patent`.`foreigncitation`', 0),
 ('`patent`.`usapplicationcitation`', 0),
 ('`patent`.`uspatentcitation`', 0),
 ('`patent`.`application`', 0),
 ('`patent`.`us_term_of_grant`', 0),
 ('`patent`.`location_assignee`', 0),
 ('`patent`.`location_inventor`', 0),
 ('`patent`.`assignee`', 0),
 ('`PatentsView`.`temp_assignee_num_patents`', 0),
 ('`patent`.`inventor`', 0),
 ('`patent`.`rawlawyer`', 0),
 ('`patent`.`lawyer`', 0),
 ('`patent`.`foreign_priority`', 0),
 ('`patent`.`pct_data`', 0),
 ('`patent`.`cpc_current`', 0),
 ('`patent`.`cpc_subsection`', 0),
 ('`patent`.`cpc_group`', 0),
 ('`patent`.`cpc_subgroup`', 0),
 ('`patent`.`ipcr`', 0),
 ('`patent`.`nber`', 0),
 ('`patent`.`nber_category`', 0),
 ('`

In [160]:
node_levels=[]
current_level=0
while True:
    current_level_items=list(filter( lambda x: x[1]==0, G.in_degree))
#    print(current_level_items)
    if len(current_level_items) <1:
        break
    for current_level_item in current_level_items:
        node_levels.append((current_level_item[0],current_level))
        G.remove_node(current_level_item[0])
    current_level+=1
#    print("---")
    

In [179]:
nx.clique.enumerate_all_cliques(G)

NetworkXNotImplemented: not implemented for directed type

In [174]:
pd.DataFrame(list(G.edges())).to_csv("reporting_db_generation_dependencies.csv", index=False, header=False)

In [163]:
pd.DataFrame(node_levels).to_csv("table_levels.csv", index=False, header=False)